<a href="https://colab.research.google.com/github/alexandergribenchenko/Data_Science_Toolkit/blob/main/Optimization/DS_Opt_02_NB_Pulp_Aeronautic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A. Librerías

In [ ]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from pulp import *

# 01. Problema Ejemplo 01

## 01.01.Planteamiento

[Fuente del problema](https://www.ingenieriaindustrialonline.com/investigacion-de-operaciones/ejercicios-de-programacion-lineal/)

Un herrero con 80 Kg de acero y 120 Kg de aluminio quiere hacer bicicletas de paseo y de montaña que quiere vender, respectivamente a 20.000 y 15.000 pesos cada una para sacar el máximo beneficio. Para la de paseo empleará 1 Kg De acero y 3 Kg de aluminio, y para la de montaña 2 Kg de ambos metales. La cantida total de acero con que cuenta es 80 Kg y la cantidad total de Aluminio es 120 Kg. 

¿Cuántas bicicletas de paseo y de montaña deberá fabricar para maximizar las utilidades?

|	|Requerimiento de materiales| |		Precio de venta|
|-|:----:|-|-|
|	              |Acero (Kg)|	Aluminio (Kg)|	
|Bicicleta paseo  |	1        |	3|	20000|
|Bicicleta montaña|	2        |	2|	15000|
|Total material   |	80       |	120|	 |

#### Definición de las variables
- **X** = Cantidad de bicicletas de paseo a producir
- **Y** = Cantidad de bicicletas de montaña a producir

#### Función objetivo

- Zmax = 20000X + 15000Y

#### Restricciones

- X + 2Y <= 80 (Disponibilidad de acero)
- 3X + 2Y <= 120 (Disponibilidad de aluminio)
- X; Y >= 0 (Restricciones de NO negatividad)

In [ ]:
problema_bicicletas = LpProblem('Bicicletas', LpMaximize)
problema_bicicletas

Bicicletas:
MAXIMIZE
None
VARIABLES

In [ ]:
origen = ['bic_paseo','bic_montaña']
destino = ['venta'] 

In [ ]:
rutas = [(i,j) for i in origen for j in destino] ### indicamos las rutas (aristas) del modelo
rutas

[('bic_paseo', 'venta'), ('bic_montaña', 'venta')]

In [ ]:
cantidad = LpVariable.dicts('Cantidad', (origen, destino),0)
cantidad

{'bic_paseo': {'venta': Cantidad_bic_paseo_venta},
 'bic_montaña': {'venta': Cantidad_bic_montaña_venta}}

In [ ]:
origen

['bic_paseo', 'bic_montaña']

In [ ]:
valor_venta = {'bic_paseo':{'venta': 20000},
               'bic_montaña':{'venta': 15000}}
valor_venta

{'bic_paseo': {'venta': 20000}, 'bic_montaña': {'venta': 15000}}

In [ ]:
problema_bicicletas += lpSum(cantidad[i][j]*valor_venta[i][j] for (i,j) in rutas)
problema_bicicletas

Bicicletas:
MAXIMIZE
15000*Cantidad_bic_montaña_venta + 20000*Cantidad_bic_paseo_venta + 0
VARIABLES
Cantidad_bic_montaña_venta Continuous
Cantidad_bic_paseo_venta Continuous

In [ ]:
acero_total = {'venta': 80}

In [ ]:
aluminio_total = {'venta': 120}

In [ ]:
destino

['venta']

In [ ]:
cantidad

{'bic_paseo': {'venta': Cantidad_bic_paseo_venta},
 'bic_montaña': {'venta': Cantidad_bic_montaña_venta}}

In [ ]:
demanda_acero = {'bic_paseo': 1 , 'bic_montaña': 2}
demanda_acero

{'bic_paseo': 1, 'bic_montaña': 2}

In [ ]:
demanda_aluminio = {'bic_paseo': 3 , 'bic_montaña': 2}
demanda_aluminio

{'bic_paseo': 3, 'bic_montaña': 2}

In [ ]:
for j in destino:
    problema_bicicletas += lpSum(cantidad[i][j]*demanda_acero[i] for i in origen) == acero_total[j]

for j in destino:
    problema_bicicletas += lpSum(cantidad[i][j]*demanda_aluminio[i] for i in origen) == aluminio_total[j]
    
problema_bicicletas

Bicicletas:
MAXIMIZE
15000*Cantidad_bic_montaña_venta + 20000*Cantidad_bic_paseo_venta + 0
SUBJECT TO
_C1: 2 Cantidad_bic_montaña_venta + Cantidad_bic_paseo_venta = 80

_C2: 2 Cantidad_bic_montaña_venta + 3 Cantidad_bic_paseo_venta = 120

VARIABLES
Cantidad_bic_montaña_venta Continuous
Cantidad_bic_paseo_venta Continuous

In [ ]:
### Resolvemos e imprimimos el estatus de la solución
problema_bicicletas.solve()
print("Status:", LpStatus[problema_bicicletas.status])

Status: Optimal


In [ ]:
### Imprimamos la solución
for v in problema_bicicletas.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)
print('El costo maximo es:', value(problema_bicicletas.objective))

Cantidad_bic_montaña_venta = 30.0
Cantidad_bic_paseo_venta = 20.0
El costo maximo es: 850000.0


# 02. Problema Ejemplo 02

MG Auto has three plants in Los Angeles, Detroit, and New Orleans, and two major distribution centers in Denver and Miami. The capacities of the three plants during the next quarter are 1000, 1500, and 1200 cars. The quarterly demands at the two distribution centers are 2300 and 1400 cars.

The trucking company in charge of transporting the cars charges 8 cents per mile per car. The transportation costs per car on the different routes, rounded to the closest dollar, are given in the table below.

MG Auto wants to determine how many cars to transport from each plant to its distribution centers while minimizing the total cost to transport these cars.

1. What is the number of car shipped from Detroit to Denver?

2. What is the optimal objective value?

In [ ]:
origen = ['LA', 'Detroit','New Orleans']
destino = ['Denver', 'Miami']

oferta = {'LA': 1000, 'Detroit': 1500, 'New Orleans': 1200}
demanda = {'Denver': 2300, 'Miami': 1400}

costo_envio = {'LA':{'Denver': 80, 'Miami': 215},
                'Detroit':{'Denver': 100, 'Miami': 108},
                'New Orleans': {'Denver':102, 'Miami':68}}

In [ ]:
prob = LpProblem('Transporte', LpMinimize) ### Buscamos minimizar los costos

In [ ]:
type(prob)

pulp.pulp.LpProblem

In [ ]:
prob

Transporte:
MINIMIZE
None
VARIABLES

In [ ]:
rutas = [(i,j) for i in origen for j in destino] ### indicamos las rutas (aristas) del modelo
rutas

[('LA', 'Denver'),
 ('LA', 'Miami'),
 ('Detroit', 'Denver'),
 ('Detroit', 'Miami'),
 ('New Orleans', 'Denver'),
 ('New Orleans', 'Miami')]

In [ ]:
cantidad = LpVariable.dicts('Cantidad de Envio', (origen, destino),0)
cantidad

{'LA': {'Denver': Cantidad_de_Envio_LA_Denver,
  'Miami': Cantidad_de_Envio_LA_Miami},
 'Detroit': {'Denver': Cantidad_de_Envio_Detroit_Denver,
  'Miami': Cantidad_de_Envio_Detroit_Miami},
 'New Orleans': {'Denver': Cantidad_de_Envio_New_Orleans_Denver,
  'Miami': Cantidad_de_Envio_New_Orleans_Miami}}

In [ ]:
costo_envio = {'LA':{'Denver': 80, 'Miami': 215},
                'Detroit':{'Denver': 100, 'Miami': 108},
                'New Orleans': {'Denver':102, 'Miami':68}}

In [ ]:
type(cantidad['LA']['Denver'])

pulp.pulp.LpVariable

In [ ]:
costo_envio

{'LA': {'Denver': 80, 'Miami': 215},
 'Detroit': {'Denver': 100, 'Miami': 108},
 'New Orleans': {'Denver': 102, 'Miami': 68}}

In [ ]:
variable_01 = lpSum(cantidad[i][j]*costo_envio[i][j] for (i,j) in rutas)
variable_01

100*Cantidad_de_Envio_Detroit_Denver + 108*Cantidad_de_Envio_Detroit_Miami + 80*Cantidad_de_Envio_LA_Denver + 215*Cantidad_de_Envio_LA_Miami + 102*Cantidad_de_Envio_New_Orleans_Denver + 68*Cantidad_de_Envio_New_Orleans_Miami + 0

In [ ]:
prob += variable_01
prob

Transporte:
MINIMIZE
100*Cantidad_de_Envio_Detroit_Denver + 108*Cantidad_de_Envio_Detroit_Miami + 80*Cantidad_de_Envio_LA_Denver + 215*Cantidad_de_Envio_LA_Miami + 102*Cantidad_de_Envio_New_Orleans_Denver + 68*Cantidad_de_Envio_New_Orleans_Miami + 0
VARIABLES
Cantidad_de_Envio_Detroit_Denver Continuous
Cantidad_de_Envio_Detroit_Miami Continuous
Cantidad_de_Envio_LA_Denver Continuous
Cantidad_de_Envio_LA_Miami Continuous
Cantidad_de_Envio_New_Orleans_Denver Continuous
Cantidad_de_Envio_New_Orleans_Miami Continuous

In [ ]:
prob = LpProblem('Transporte', LpMinimize)
cantidad = LpVariable.dicts('Cantidad de Envio', (origen, destino),0)

prob += lpSum(cantidad[i][j]*costo_envio[i][j] for (i,j) in rutas)
prob

Transporte:
MINIMIZE
100*Cantidad_de_Envio_Detroit_Denver + 108*Cantidad_de_Envio_Detroit_Miami + 80*Cantidad_de_Envio_LA_Denver + 215*Cantidad_de_Envio_LA_Miami + 102*Cantidad_de_Envio_New_Orleans_Denver + 68*Cantidad_de_Envio_New_Orleans_Miami + 0
VARIABLES
Cantidad_de_Envio_Detroit_Denver Continuous
Cantidad_de_Envio_Detroit_Miami Continuous
Cantidad_de_Envio_LA_Denver Continuous
Cantidad_de_Envio_LA_Miami Continuous
Cantidad_de_Envio_New_Orleans_Denver Continuous
Cantidad_de_Envio_New_Orleans_Miami Continuous

In [ ]:
prob = LpProblem('Transporte', LpMinimize)
prob

Transporte:
MINIMIZE
None
VARIABLES

In [ ]:
for j in destino:
    prob += lpSum(cantidad[i][j] for i in origen) == demanda[j]
prob

Transporte:
MINIMIZE
None
SUBJECT TO
_C1: Cantidad_de_Envio_Detroit_Denver + Cantidad_de_Envio_LA_Denver
 + Cantidad_de_Envio_New_Orleans_Denver = 2300

_C2: Cantidad_de_Envio_Detroit_Miami + Cantidad_de_Envio_LA_Miami
 + Cantidad_de_Envio_New_Orleans_Miami = 1400

VARIABLES
Cantidad_de_Envio_Detroit_Denver Continuous
Cantidad_de_Envio_Detroit_Miami Continuous
Cantidad_de_Envio_LA_Denver Continuous
Cantidad_de_Envio_LA_Miami Continuous
Cantidad_de_Envio_New_Orleans_Denver Continuous
Cantidad_de_Envio_New_Orleans_Miami Continuous

In [ ]:
for i in origen:
    prob += lpSum(cantidad[i][j] for j in destino) <= oferta[i]

In [ ]:
prob

Transporte:
MINIMIZE
None
SUBJECT TO
_C1: Cantidad_de_Envio_Detroit_Denver + Cantidad_de_Envio_LA_Denver
 + Cantidad_de_Envio_New_Orleans_Denver = 2300

_C2: Cantidad_de_Envio_Detroit_Miami + Cantidad_de_Envio_LA_Miami
 + Cantidad_de_Envio_New_Orleans_Miami = 1400

_C3: Cantidad_de_Envio_LA_Denver + Cantidad_de_Envio_LA_Miami <= 1000

_C4: Cantidad_de_Envio_Detroit_Denver + Cantidad_de_Envio_Detroit_Miami
 <= 1500

_C5: Cantidad_de_Envio_New_Orleans_Denver
 + Cantidad_de_Envio_New_Orleans_Miami <= 1200

VARIABLES
Cantidad_de_Envio_Detroit_Denver Continuous
Cantidad_de_Envio_Detroit_Miami Continuous
Cantidad_de_Envio_LA_Denver Continuous
Cantidad_de_Envio_LA_Miami Continuous
Cantidad_de_Envio_New_Orleans_Denver Continuous
Cantidad_de_Envio_New_Orleans_Miami Continuous

In [ ]:
### Resolvemos e imprimimos el estatus de la solución
prob.solve()
print("Status:", LpStatus[prob.status])

Status: Optimal


In [ ]:
### Imprimamos la solución
for v in prob.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)
print('El costo mínimo es:', value(prob.objective))

Cantidad_de_Envio_Detroit_Denver = 1100.0
Cantidad_de_Envio_Detroit_Miami = 400.0
Cantidad_de_Envio_LA_Miami = 1000.0
Cantidad_de_Envio_New_Orleans_Denver = 1200.0


TypeError: ignored